In [6]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 비교군 날짜 설정 start_day, mid_day, end_day
start_day = datetime(2024, 9, 1)
mid_day = datetime(2024, 10, 1)
end_day = datetime(2024, 11, 1)

start_date = start_day.strftime('%Y-%m-%d')
temp_date = mid_day - pd.Timedelta(days=1)
mid_date1 = temp_date.strftime('%Y-%m-%d')
mid_date2 = mid_day.strftime('%Y-%m-%d')
temp_date = end_day - pd.Timedelta(days=1)
end_date = temp_date.strftime('%Y-%m-%d')

# rate function
def rate(before, after):
    data = (after - before) / before * 100
    data = np.round(data, 1)
    return data

# Read Dataframe and fine data
user_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_pb_user_export_2024-11-04_112746.csv')
auth_df = pd.read_csv('/Users/seanyu_mac/Downloads/public_pb_user_auth_info_export_2024-11-04_112813.csv')

user_df.rename(columns={'id': 'key'}, inplace=True)
auth_df.rename(columns={'user_id': 'key'}, inplace=True)

df = pd.merge(user_df[['key', 'email', 'reg_dt', 'last_login_dt', 'del_dt']], auth_df[['key', 'email', 'sex', 'updated_at', 'is_delete', 'birthday', 'sns_type']], on='key', how='left')

pre_col_list = df.columns.tolist()
post_col_list = ['key', 'email_x', 'email_y', 'reg_dt', 'last_login_dt', 'updated_at', 'del_dt', 'is_delete', 'sex', 'birthday', 'sns_type']
df = df[post_col_list]
df.rename(columns={'reg_dt': 'first_seen', 'last_login_dt': 'last_seen_x', 'updated_at': 'last_seen_y'}, inplace=True)

del df['last_seen_y']
df.rename(columns={'last_seen_x': 'last_seen'}, inplace=True)

# read consult_df and groupby on monthly basis
consult_df = pd.read_excel('/Users/seanyu_mac/Downloads/statics_result_20241104_122338.xlsx')
consult_df = consult_df[consult_df['년도'] == 2024]
temp_df = consult_df.groupby('월').sum()
temp_df['num'] = temp_df['1:1 상담건수'] + temp_df['사진 분석 건수'] 

# total_sign_up_num, pre_sign_up_num, post_sign_up_num
dates = []

for i in df['first_seen'].tolist():
    strftime = i[:10]
    date = pd.to_datetime(strftime)
    dates.append(date)

df['first_seen'] = dates
pre_con = (df['first_seen'] >= start_day) & (df['first_seen'] < mid_day)
post_con = (df['first_seen'] >= mid_day) & (df['first_seen'] < end_day)
all_con = df['first_seen'] < end_day

total_sign_up_num = len(df[all_con].index)
pre_sign_up_num = len(df[pre_con].index)
post_sign_up_num = len(df[post_con].index)

# total_consult_num, pre_consult_num, post_consult_num
last_month = mid_day.strftime('%m')
last_month = int(last_month)
total_consult_num = 0

for i in range(last_month):
    i = i + 1
    total_consult_num = total_consult_num + temp_df['num'][i]

pre_consult_num = temp_df['num'][last_month - 1]
post_consult_num = temp_df['num'][last_month]


In [7]:
print('월별 상담수 추이')
temp_df['num']

월별 상담수 추이


월
1     2308
2     1978
3     3386
4     2817
5     2658
6     2734
7     2814
8     2228
9     1982
10    2200
11     181
Name: num, dtype: int64

In [8]:
sign_up_df = df[df['first_seen'] >= datetime(2024, 1, 1)]
sign_up_df['month'] = sign_up_df['first_seen'].astype(str)
sign_months = []
for i in sign_up_df['month']:
    i = i[5:7]
    sign_months.append(i)
sign_up_df['month'] = sign_months
print('월별 회원가입자수 추이')
sign_up_df.groupby('month').size()

월별 회원가입자수 추이


/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_3981/3493840013.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_up_df['month'] = sign_up_df['first_seen'].astype(str)
/var/folders/wl/txjbcgld2sngchsc7txxqppm0000gp/T/ipykernel_3981/3493840013.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_up_df['month'] = sign_months


month
01    2413
02    1297
03    2281
04    1839
05    2009
06    1988
07    2075
08    1495
09    1169
10    1560
11     185
dtype: int64

In [12]:
print(f'1. 지지난달`({start_date} ~ {mid_date1})`과 지난달`({mid_date2} ~ {end_date})`을 비교')
print(f'2. {end_date}일 기준 총 회원 가입 유저 수는 {total_sign_up_num}명')
print(f'3. {start_day.strftime('%m')}월 회원 가입자 수: {pre_sign_up_num}명 / {mid_day.strftime('%m')}월 회원 가입자 수: {post_sign_up_num}명으로 전월 대비 {rate(pre_sign_up_num, post_sign_up_num)}% 증감')
print(f'4. {mid_day.strftime('%m')}월 기준, 총 누적 상담 건수는 {total_consult_num}건')
print(f'5. {start_day.strftime('%m')}월 상담 수: {pre_consult_num}건, {mid_day.strftime('%m')}월 상담 수: {post_consult_num}건으로 전월 대비 {rate(pre_consult_num, post_consult_num)}% 증감')
print(f'6. 평균 DAU 656명 (지난달 651명 대비 {rate(651, 656)}% 증가)')
print(f'7. 평균 MAU 10511명 (지난달 9882명 대비 {rate(9882, 10511)}% 증가)')

1. 지지난달`(2024-09-01 ~ 2024-09-30)`과 지난달`(2024-10-01 ~ 2024-10-31)`을 비교
2. 2024-10-31일 기준 총 회원 가입 유저 수는 24385명
3. 09월 회원 가입자 수: 1169명 / 10월 회원 가입자 수: 1560명으로 전월 대비 33.4% 증감
4. 10월 기준, 총 누적 상담 건수는 25105건
5. 09월 상담 수: 1982건, 10월 상담 수: 2200건으로 전월 대비 11.0% 증감
6. 평균 DAU 656명 (지난달 651명 대비 0.8% 증가)
7. 평균 MAU 10511명 (지난달 9882명 대비 6.4% 증가)


In [14]:
# 8월 이후 카카오랑 친한스토어 매출파일 Read
kakao_df = pd.read_excel('/Users/seanyu_mac/Downloads/20241104152401.xlsx')
chinhan_df1 = pd.read_excel('/Users/seanyu_mac/Downloads/report.xlsx')
chinhan_df2 = pd.read_excel('/Users/seanyu_mac/Downloads/report (1).xlsx')

kakao_df[kakao_df['취소완료일'].isna() == False].index

kakao_df.drop(kakao_df[kakao_df['취소완료일'].isna() == False].index, inplace=True)
kakao_df['date'] = kakao_df['발송요청일']
kakao_df['id'] = kakao_df['결제번호']
kakao_df['name'] = kakao_df['상품명']
kakao_df['quantity'] = kakao_df['수량']
kakao_df['sales'] = kakao_df['정산기준금액']

chinhan_df = pd.concat([chinhan_df1, chinhan_df2])
chinhan_df.drop(chinhan_df[chinhan_df['취소 완료일'].isna() == False].index, inplace=True)
chinhan_df['date'] = chinhan_df['결제 완료일']
chinhan_df['id'] = chinhan_df['결제 번호']
chinhan_df['name'] = chinhan_df['옵션명']
chinhan_df['quantity'] = chinhan_df['수량']
chinhan_df['sales'] = chinhan_df['정산가']

kakao_df = kakao_df[['date', 'id', 'name' , 'quantity', 'sales']]
kakao_df.drop(kakao_df[kakao_df['date'].isna()].index, inplace=True)
chinhan_df = chinhan_df[['date', 'id', 'name' , 'quantity', 'sales']]
sales_df = pd.concat([kakao_df, chinhan_df])

sales_df['date'] = sales_df['date'].astype(str)
sales_dates = []
for i in sales_df['date']:
    i = i[:10]
    sales_dates.append(i)
sales_df['date'] = sales_dates
sales_df['date'] = pd.to_datetime(sales_df['date'])
sales_df['month'] = sales_df['date'].dt.strftime('%m')
sales_df.sort_values(by='month', ascending=True, inplace=True)
sales_df.groupby('month')['sales'].sum()

/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


month
01     4095420
02     7531040
03    10053580
04    13277500
05     9880280
06     9561430
07    11381060
08     9907540
09    10630040
10     8953640
11     1721730
Name: sales, dtype: int64

In [18]:
print(f'1. 총 {end_day - sales_df['date'].min()}일 동안 누적 판매금액 : {sales_df['sales'].sum():,}원')
print(f'2. 일 평균 판매액 : {sales_df['sales'].sum() / len(sales_df['date'].unique())}원')
print(f'3. 객단가 : {sales_df['sales'].sum() / len(sales_df['id'].unique())}원')
print(f'4. 9월 매출 10,630,040원, 10월 매출 8,953,640원으로 {rate(10630040, 8953640)}% 증감')
print(f'5. 현재 상담 추천 제품 판매 관련 앱 리빌드 과정으로 정확한 전환율 측정 불가 (기존의 약 25% 내외 유지 중)')

1. 총 289 days 00:00:00일 동안 누적 판매금액 : 96,993,260원
2. 일 평균 판매액 : 332168.69863013696원
3. 객단가 : 51674.61907298881원
4. 9월 매출 10,630,040원, 10월 매출 8,953,640원으로 -15.8% 증감
5. 현재 상담 추천 제품 판매 관련 앱 리빌드 과정으로 정확한 전환율 측정 불가 (기존의 약 25% 내외 유지 중)


292